# Applying Pre-Implemented Analyses

An *Analysis* class can be applied to a dataset in a flexible manner, such as the parameterisation how and where the data is stored, which derivatives are required, and the computing environment in which to generate the derivatives.

## Inspecting the Analysis Class

We will start by importing a predefined Analysis class from `example.analysis`, which performs the same analysis as the workflow in the [Workflows Notebook](basic_workflow.ipynb). We print the "menu", the list of inputs, derivatives and parameters objects of this class can receive/derive, using the `static_menu` class method.

In [1]:
from example.analysis import BasicBrainAnalysis
print(BasicBrainAnalysis.static_menu())


example.analysis.BasicBrainAnalysis Menu 
-----------------------------------------

Inputs:
    magnitude : nifti_gz
        A magnitude image (e.g. T1w, T2w, etc..)

Outputs:
    brain : nifti_gz
        Skull-stripped magnitude image
    brain_mask : nifti_gz
        Brain mask used for skull-stripping
    smooth : nifti_gz
        Smoothed magnitude image
    smooth_masked : nifti_gz
        Smoothed and masked magnitude image

Parameters:
    smoothing_fwhm : float
        The full-width-half-maxium radius of the smoothing kernel


## Defining the Dataset to Analyse

Arcana implicitly handles a lot of the menial tasks involved with data input/outputs such as file format conversions and inserting/retrieving data from a repository service (e.g. XNAT). To specify where your data is you need to create a Dataset object.

The simplest form of dataset object is just a directory on (or mounted on) your local file system with data for one session stored within it, e.g.

In [2]:
%%bash
# Create a dataset for a single session in a flat directory. We will copy data from the BIDS formatted ds000114
SAMPLE_DSET=data/sample-datasets/depth0
mkdir -p $SAMPLE_DSET
find data/ds000114/sub-01/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/
tree $SAMPLE_DSET

data/sample-datasets/depth0
├── sub-01_ses-test_T1w.nii.gz
├── sub-01_ses-test_T1w_bet.nii.gz
├── sub-01_ses-test_dwi.nii.gz
├── sub-01_ses-test_task-covertverbgeneration_bold.nii.gz
├── sub-01_ses-test_task-fingerfootlips_bold.nii.gz
├── sub-01_ses-test_task-linebisection_bold.nii.gz
├── sub-01_ses-test_task-overtverbgeneration_bold.nii.gz
└── sub-01_ses-test_task-overtwordrepetition_bold.nii.gz

0 directories, 8 files


In [22]:
from arcana import Dataset
dset0 = Dataset('data/sample-datasets/depth0')
print(dset0)

Dataset(name='/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/data/sample-datasets/depth0', depth=0, repository=LocalFileSystemRepo())


Notice the `depth` of this dataset is `0`. This means that there aren't any sub-directories for separate subjects or visits in it. However, all datasets in Arcana have an implicit depth of 2 (although future versions may relax this restriction) so we can see that the single "session" (a single visit of a subject) is assigned default subject and visit IDs of 'SUBJECT' and 'VISIT' respectively.

In [23]:
print('subjects:', list(dset0.subject_ids))
print('visits:', list(dset0.visit_ids))

['SUBJECT']
['VISIT']


For a multi-subject dataset we can add sub-directories for each subject

In [17]:
%%bash
# Create a dataset for a multiple subjects in separate sub-directories by copying data from the BIDS formatted ds000114
SAMPLE_DSET=data/sample-datasets/depth1
mkdir -p $SAMPLE_DSET/sub1 $SAMPLE_DSET/sub2  $SAMPLE_DSET/sub3
find data/ds000114/sub-01/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub1
find data/ds000114/sub-02/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub2
find data/ds000114/sub-03/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub3
tree $SAMPLE_DSET

data/sample-datasets/depth1
├── sub1
│   ├── sub-01_ses-test_T1w.nii.gz
│   ├── sub-01_ses-test_T1w_bet.nii.gz
│   ├── sub-01_ses-test_dwi.nii.gz
│   ├── sub-01_ses-test_task-covertverbgeneration_bold.nii.gz
│   ├── sub-01_ses-test_task-fingerfootlips_bold.nii.gz
│   ├── sub-01_ses-test_task-linebisection_bold.nii.gz
│   ├── sub-01_ses-test_task-overtverbgeneration_bold.nii.gz
│   └── sub-01_ses-test_task-overtwordrepetition_bold.nii.gz
├── sub2
│   ├── sub-02_ses-test_T1w.nii.gz
│   ├── sub-02_ses-test_dwi.nii.gz
│   ├── sub-02_ses-test_task-covertverbgeneration_bold.nii.gz
│   ├── sub-02_ses-test_task-fingerfootlips_bold.nii.gz
│   ├── sub-02_ses-test_task-linebisection_bold.nii.gz
│   ├── sub-02_ses-test_task-overtverbgeneration_bold.nii.gz
│   └── sub-02_ses-test_task-overtwordrepetition_bold.nii.gz
└── sub3
    ├── sub-03_ses-test_T1w.nii.gz
    ├── sub-03_ses-test_dwi.nii.gz
    ├── sub-03_ses-test_task-covertverbgeneration_bold.nii.gz
    ├── sub-03_ses-test_task-fingerfootlips_

In [27]:
dset1 = Dataset('data/sample-datasets/depth1', depth=1)
print(dset1)
print('subjects:', list(dset1.subject_ids))
print('visits:', list(dset1.visit_ids))

Dataset(name='/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/data/sample-datasets/depth1', depth=1, repository=LocalFileSystemRepo())
subjects: ['sub1', 'sub2', 'sub3']
visits: ['VISIT']


**Note** that we need to explicitly provide the depth of `1` otherwise Arcana will interpret our 'sub1', 'sub2' and 'sub3' as filesets.

In [31]:
%%bash
# Create a dataset for a multiple subjects in separate sub-directories by copying data from the BIDS formatted ds000114
SAMPLE_DSET=data/sample-datasets/depth2
mkdir -p $SAMPLE_DSET/sub1/test $SAMPLE_DSET/sub1/retest $SAMPLE_DSET/sub2/test $SAMPLE_DSET/sub2/test \
         $SAMPLE_DSET/sub2/retest $SAMPLE_DSET/sub3/test $SAMPLE_DSET/sub3/retest
find data/ds000114/sub-01/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub1/test
find data/ds000114/sub-02/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub2/test
find data/ds000114/sub-03/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub3/test
find data/ds000114/sub-01/ses-retest -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub1/retest
find data/ds000114/sub-02/ses-retest -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub2/retest
find data/ds000114/sub-03/ses-retest -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub3/retest
tree $SAMPLE_DSET

data/sample-datasets/depth2
├── sub1
│   ├── retest
│   │   ├── sub-01_ses-retest_T1w.nii.gz
│   │   ├── sub-01_ses-retest_dwi.nii.gz
│   │   ├── sub-01_ses-retest_task-covertverbgeneration_bold.nii.gz
│   │   ├── sub-01_ses-retest_task-fingerfootlips_bold.nii.gz
│   │   ├── sub-01_ses-retest_task-linebisection_bold.nii.gz
│   │   ├── sub-01_ses-retest_task-overtverbgeneration_bold.nii.gz
│   │   └── sub-01_ses-retest_task-overtwordrepetition_bold.nii.gz
│   └── test
│       ├── sub-01_ses-test_T1w.nii.gz
│       ├── sub-01_ses-test_T1w_bet.nii.gz
│       ├── sub-01_ses-test_dwi.nii.gz
│       ├── sub-01_ses-test_task-covertverbgeneration_bold.nii.gz
│       ├── sub-01_ses-test_task-fingerfootlips_bold.nii.gz
│       ├── sub-01_ses-test_task-linebisection_bold.nii.gz
│       ├── sub-01_ses-test_task-overtverbgeneration_bold.nii.gz
│       └── sub-01_ses-test_task-overtwordrepetition_bold.nii.gz
├── sub2
│   ├── retest
│   │   ├── sub-02_ses-retest_T1w.nii.gz
│   │   ├── sub-02_ses-rete

In [32]:
dset2 = Dataset('data/sample-datasets/depth2', depth=2)
print(dset2)
print('subjects:', list(dset2.subject_ids))
print('visits:', list(dset2.visit_ids))

Dataset(name='/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/data/sample-datasets/depth2', depth=2, repository=LocalFileSystemRepo())
subjects: ['sub1', 'sub2', 'sub3']
visits: ['retest', 'test']


However, just say the `retest` session of `Subject 3` was corrupted we could exclude it from the analysis by either dropping `Subject 3` or `retest` from the dataset by filtering the IDs

In [33]:
dset2_filter_subs = Dataset('data/sample-datasets/depth2', depth=2, subject_ids=['sub1', 'sub2'])
print(dset2_filter_subs)
print('subjects:', list(dset2_filter_subs.subject_ids))
print('visits:', list(dset2_filter_subs.visit_ids))

Dataset(name='/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/data/sample-datasets/depth2', depth=2, repository=LocalFileSystemRepo())
subjects: ['sub1', 'sub2']
visits: ['retest', 'test']


In [34]:
dset2_filter_vis = Dataset('data/sample-datasets/depth2', depth=2, visit_ids=['test'])
print(dset2_filter_vis)
print('subjects:', list(dset2_filter_vis.subject_ids))
print('visits:', list(dset2_filter_vis.visit_ids))

Dataset(name='/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/data/sample-datasets/depth2', depth=2, repository=LocalFileSystemRepo())
subjects: ['sub1', 'sub2', 'sub3']
visits: ['test']


In [35]:
dset2_filter_both = Dataset('data/sample-datasets/depth2', depth=2, subject_ids=['sub1', 'sub2'], visit_ids=['test'])
print(dset2_filter_both)
print('subjects:', list(dset2_filter_both.subject_ids))
print('visits:', list(dset2_filter_both.visit_ids))

Dataset(name='/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/data/sample-datasets/depth2', depth=2, repository=LocalFileSystemRepo())
subjects: ['sub1', 'sub2']
visits: ['test']
